### RatU rollipram SD recording info

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from neuropy import analyses 
from neuropy import plotting
import subjects

sess = subjects.sdrol.ratU_2021_08_09[0]
print(sess.recinfo)

### Set probe configuration
- This has been copied from RatUDay4SD as the recording info and probegroup are the same.
- RatU_Day4SD has two probes: both 128chan-8shanks diagnostic biochips.
- 64 channels in one of the probes (implanted in left hemisphere) had no signal from 4 shanks (probably one of the intan chips was faulty). So only 192 channels were recorded, the channels in .dat file are already order according to depth.


In [ ]:
%matplotlib inline
from neuropy.core import Shank, Probe, ProbeGroup
from neuropy.plotting import plot_probe

shanks = []
channel_groups = sess.recinfo.channel_groups
badchans = sess.recinfo.skipped_channels

#--- diagnostic-biochip 8 shanks -----------
for i in range(8):
    chans = channel_groups[i]
    shank = Shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0,-7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)

probe1 = Probe(shanks)

#--- dignostic biochip 4 shanks (4 shanks were bad) ----------- 
shanks = []
for i in range(8,12):

    shank = shank.auto_generate(
        columns=2,
        contacts_per_column=8,
        xpitch=15,
        ypitch=15,
        y_shift_per_column=[0, -7.5],
        channel_id=np.append(
            channel_groups[i][::2][::-1], channel_groups[i][1::2][::-1]
        ),
    )
    shank.set_disconnected_channels(sess.recinfo.skipped_channels)
    shanks.append(shank)


probe2 = Probe(shanks)
probe2.move((probe1.x_max+500,0))

prbgrp = ProbeGroup()
prbgrp.add_probe(probe1)
prbgrp.add_probe(probe2)

prbgrp.save(sess.filePrefix.with_suffix(".probegroup.npy"))

plot_probe(prbgrp)

### Experimental paradigm
The epoch timestamps were manually entered using Jahangir's original .epoch.npy file. He had labelled 'recovery sleep' period as 'post'. However, according to my convention all non-maze time periods following first maze exposure is post.

In [ ]:
import pandas as pd
from neuropy.core import Epoch

epochs = pd.DataFrame(
    {
        "start": [0,8727,12707,30777,12707],
        "stop": [8726,12706,30776,48844,48844],
        "label": ["pre", "maze", "sd",'rs','post'],
    }
)

paradigm = Epoch(epochs=epochs)
paradigm.save(sess.filePrefix.with_suffix(".paradigm.npy"))

### Artifacts
A typical session will have some artifacts that may negatively influence many analyses. Using a simple zscore measure, we can identify epochs where signal is above some threshold.

In [ ]:
signal = sess.eegfile.get_signal([20,170])
artifact_epochs = analyses.detect_artifact_epochs(signal, thresh=7,edge_cutoff=2)
artifact_epochs.save(sess.filePrefix.with_suffix(".artifact.npy"))

In [ ]:
sess.recinfo.write_epochs(sess.artifact)

In [ ]:
signal = sess.eegfile.get_signal([20])
plotting.plot_artifact_epochs(sess.artifact, signal)

### Neurons
- Jahangir had already extracted clusters into .neurons.npy. However, some of the units have neuron_type labelled as '1'. I checked their autocorrelograms and based on their shape and frate all seemed like pyramidal neurons. I am re-labelling these units as 'pyr'. There was only 1 such unit.

In [ ]:
#autocorrelogram generation for unit labelled as '1'

from neuropy.utils import neurons_util

neurons = sess.neurons.get_neuron_type(['1'])
frate = neurons.firing_rate
acg = neurons_util.calculate_neurons_acg(neurons=neurons,window_size=0.1)

_,axs = plt.subplots(5,6)
axs = axs.reshape(-1)
for i,ax in enumerate(axs):
    ax.plot(acg[i,:])
    ax.set_title(frate[i].round(2))

In [ ]:
from neuropy.core import Neurons

og_neuron_type = sess.neurons.neuron_type
new_neuron_type = np.where(og_neuron_type==['1'],'pyr',og_neuron_type)

neurons = Neurons(
    np.array(sess.neurons.spiketrains, dtype=object),
    t_stop=sess.eegfile.duration,
    sampling_rate=sess.neurons.sampling_rate,
    peak_channels=sess.neurons.peak_channels,
    waveforms=sess.neurons.waveforms,
    shank_ids=sess.neurons.shank_ids,
    neuron_type=og_neuron_type,
)
neurons.save(sess.filePrefix.with_suffix(".neurons"))